The methods 
We will be using sqaured error as our cost function.

$$
V(s) \leftarrow  V(s) + \alpha \left( G_s - V(s) \right)
$$

kNN is simple.  Once a metric is chosen, you examine all points in the training set to determine which k are closest to the test point.  The choose the label most prevelent among those k.

In [36]:
import numpy as np
from sortedcontainers import SortedList



#Defining the class to have fit and predict functions like scikit-learn
class KNN(object):
    def __init__(self,k):
        self.k=k
        
    def fit(self,x,y):
        self.x=x
        self.y=y
        
    def predict(self,Xtest):
        y=np.zeros(len(Xtest))
        for i,x in enumerate(Xtest): #returns a count and object of the iterable
            sl=SortedList(load=self.k) #from sorted containers, a sorted list that will make it easy
            #to track distances
            for j,xt in enumerate(self.x): #this is training X
                diff=x-xt
                d=diff.dot(diff) #dot product
                if len(sl)<self.k:
                    sl.add((d,self.y[j])) #add the distance and the class
                else:
                    if d<sl[-1][0]:
                        del sl[-1]
                        sl.add((d,self.y[j])) 
                #at the end of this for loop we have the k closest elements
            votes={}
            for _,v in sl: # v is the vote for a class
                votes[v]=votes.get(v,0)+1
                max_vote=0
                max_class=-1
                for v,count in votes.items():
                    if count >max_vote:
                        max_vote=count
                        max_class=v
                        
                y[i]=max_class
        return(y)
                
    def score(self,x,y):
        p=self.predict(x)

        return( np.mean(p==y))
            
            
            
    

In [52]:
data_folder="C:/Users/craig_arl/Documents/GitHub/machine_learning_examples/mnist_csv/"
xtrn=data_folder+"Xtrain.txt"
xtest=data_folder+"Xtest.txt"
ytrn=data_folder+"label_train.txt"
ytest=data_folder+"label_test.txt"


import pandas as pd
def get_data(f_name_X,f_name_Y,limit=None):
    print("Reading in and transforming data...")
    df_x = pd.read_csv(f_name_X)
    df_y = pd.read_csv(f_name_Y)
    df=df_y.join(df_x)
   
    data = df.as_matrix()
    np.random.shuffle(data)
    X = data[:, 1:] / 255.0 # data is from 0..255
    Y = data[:, 0]
    if limit is not None:
        X, Y = X[:limit], Y[:limit]
    return X, Y

#copied:
def get_xor():
    X = np.zeros((200, 2))
    X[:50] = np.random.random((50, 2)) / 2 + 0.5 # (0.5-1, 0.5-1)
    X[50:100] = np.random.random((50, 2)) / 2 # (0-0.5, 0-0.5)
    X[100:150] = np.random.random((50, 2)) / 2 + np.array([[0, 0.5]]) # (0-0.5, 0.5-1)
    X[150:] = np.random.random((50, 2)) / 2 + np.array([[0.5, 0]]) # (0.5-1, 0-0.5)
    Y = np.array([0]*100 + [1]*100)
    return X, Y

def get_donut():
    N = 200
    R_inner = 5
    R_outer = 10

    # distance from origin is radius + random normal
    # angle theta is uniformly distributed between (0, 2pi)
    R1 = np.random.randn(N//2) + R_inner
    theta = 2*np.pi*np.random.random(N//2)
    X_inner = np.concatenate([[R1 * np.cos(theta)], [R1 * np.sin(theta)]]).T

    R2 = np.random.randn(N//2) + R_outer
    theta = 2*np.pi*np.random.random(N//2)
    X_outer = np.concatenate([[R2 * np.cos(theta)], [R2 * np.sin(theta)]]).T

    X = np.concatenate([ X_inner, X_outer ])
    Y = np.array([0]*(N//2) + [1]*(N//2))
    return X, Y

In [53]:
x_train,y_train=get_data(xtrn,ytrn)
x_test,y_test=get_data(xtest,ytest)

Reading in and transforming data...
   0  -2.5633  -0.62572    0.9354    1.6618  -0.97179    0.3452  -0.086273  \
0  0  -3.4637  -0.88703  0.748670  1.434500  -1.15260  0.751390   0.507230   
1  0  -2.0966  -0.25018  0.042457  1.900700  -2.52000  0.776630   0.797510   
2  0  -1.6135  -0.59420  1.533200  1.466300  -0.49371  0.579620   0.018415   
3  0  -2.9356  -1.23560 -0.510630  0.046622  -0.16494 -1.468300   1.501300   
4  0  -4.0263  -0.96807  0.759960  2.724000  -0.14465 -0.077638   0.565700   

   0.87996  -0.46192    ...        0.4244  -0.30342  -0.41417  -0.57815  \
0 -0.40091  -0.12550    ...     -0.039145  -0.64153 -0.268930 -0.809140   
1  0.24791  -0.22920    ...      0.286230  -0.92112  0.130000 -0.688510   
2  0.77533  -1.65060    ...      0.786620   0.26431 -0.071850  0.062487   
3 -0.36378  -0.42991    ...      0.883280   0.87492 -0.065224 -0.576910   
4 -0.77676  -1.16410    ...      0.251960   0.14318 -0.432150 -0.205540   

     1.1077  -0.21329    0.154   0.11203   0

In [37]:
knn=KNN(5)
knn.fit(x_train,y_train)
knn.score(x_train,y_train)

Naive Bayes

[ True  True  True ...,  True  True  True]
[-1.5557   -0.056098  0.49028  ..., -1.1936   -0.41051  -2.1814  ] [-1.5557   -0.056098  0.49028  ..., -1.1936   -0.41051  -2.1814  ]


0.998999799959992

In [58]:
import numpy as np
from scipy.stats import norm 
from scipy.stats import multivariate_normal as mvn

class NaiveBayes(object):
    def fit(self, x,y,smoothing=10e-3):
        self.gaussians=dict()
        self.priors=dict()
        labels=set(y.astype(int))
        
        for c in labels:
            current_x=x[y==c]
            self.gaussians[c]={'mean':current_x.mean(axis=0),
                              'var': current_x.var(axis=0)+smoothing} #axis 0 down 1 across
            self.priors[c]=float(len(y[y==c]))/len(y)
    def score(self,x,y):
        p=self.predict(x)
        return( np.mean(p==y))
    
    def predict(self,x):
        N,D=x.shape
        K=len(self.gaussians)
        P=np.zeros((N,K)) #matrix for the probabilities of each class
        for c,g in self.gaussians.items():
         
            mean,var = g['mean'],g['var']
            P[:,c]=mvn.logpdf(x,mean=mean,cov=var)+np.log(self.priors[c])   #for each x prob it is class c
        return(np.argmax(P,axis=1))
    
    

In [59]:
x_train,y_train=get_data(xtrn,ytrn)
x_test,y_test=get_data(xtest,ytest)
print(y_train)

Reading in and transforming data...
   0  -2.5633  -0.62572    0.9354    1.6618  -0.97179    0.3452  -0.086273  \
0  0  -3.4637  -0.88703  0.748670  1.434500  -1.15260  0.751390   0.507230   
1  0  -2.0966  -0.25018  0.042457  1.900700  -2.52000  0.776630   0.797510   
2  0  -1.6135  -0.59420  1.533200  1.466300  -0.49371  0.579620   0.018415   
3  0  -2.9356  -1.23560 -0.510630  0.046622  -0.16494 -1.468300   1.501300   
4  0  -4.0263  -0.96807  0.759960  2.724000  -0.14465 -0.077638   0.565700   

   0.87996  -0.46192    ...        0.4244  -0.30342  -0.41417  -0.57815  \
0 -0.40091  -0.12550    ...     -0.039145  -0.64153 -0.268930 -0.809140   
1  0.24791  -0.22920    ...      0.286230  -0.92112  0.130000 -0.688510   
2  0.77533  -1.65060    ...      0.786620   0.26431 -0.071850  0.062487   
3 -0.36378  -0.42991    ...      0.883280   0.87492 -0.065224 -0.576910   
4 -0.77676  -1.16410    ...      0.251960   0.14318 -0.432150 -0.205540   

     1.1077  -0.21329    0.154   0.11203   0

In [60]:
model=NaiveBayes()
model.fit(x_train,y_train)
model.score(x_test,y_test)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
0
1
2
3
4
5
6
7
8
9


0.71342685370741488